In [5]:
import logging
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from pywer import wer, cer
import time
import pandas as pd
import argparse

from torch.utils.data import DataLoader


In [3]:
import torch.nn as nn
import torch.nn.functional as F

class LSTMCTC(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers, dropout_rate):
        super(LSTMCTC, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            dropout=dropout_rate,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        x, _ = self.lstm(x)
        x = self.dropout(x)  
        return self.fc(x)
    


In [4]:
import torch
from typing import List

class Alphabet:
    def __init__(self, alphabet_string = 'abcdefghijklmnopqrstuvwxyzåäö '):
        self.alphabet = list(alphabet_string)
        self.alphabet.append('<blank>')  # Adding blank token
        self.char_to_index = {char: index for index, char in enumerate(self.alphabet)}
        self.index_to_char = {index: char for index, char in enumerate(self.alphabet)}
        self.blank_index = len(self.alphabet) - 1  # Blank is the last index

    def text_to_array(self, text):
        return [self.char_to_index[char] for char in str.lower(text) if char in self.char_to_index]

    def array_to_text(self, array):
        return ''.join(self.index_to_char.get(int(index), '<UNK>') for index in array)

    def decode(self, log_probs: torch.Tensor, remove_blanks: bool = False) -> List[str]:
        """
        Decode log probabilities to text using simple greedy decoding.
        
        Args:
        log_probs (torch.Tensor): Log probabilities from the model
                                  Shape: (batch_size, sequence_length, num_classes)
        remove_blanks (bool): If True, remove all blank tokens from the output
        
        Returns:
        List[str]: Decoded texts for each item in the batch
        """
        # Get the most likely class at each step
        predictions = torch.argmax(log_probs, dim=-1)  # Shape: (batch_size, sequence_length)
        
        batch_texts = []
        for batch_item in predictions:
            text = self._decode_prediction(batch_item, remove_blanks)
            batch_texts.append(text)
        
        return batch_texts

    def _decode_prediction(self, prediction: torch.Tensor, remove_blanks: bool) -> str:
        """
        Decode a single prediction sequence to text.
        
        Args:
        prediction (torch.Tensor): Prediction sequence for a single item
                                   Shape: (sequence_length,)
        remove_blanks (bool): If True, remove all blank tokens from the output
        
        Returns:
        str: Decoded text
        """
        decoded = []
        previous = None
        for p in prediction:
            p = p.item()
            if remove_blanks:
                if p != self.blank_index and p != previous:
                    if p < len(self.alphabet) - 1:  # Exclude blank token
                        decoded.append(self.alphabet[p])
            else:
                if p != previous:
                    if p < len(self.alphabet):  # Include blank token
                        decoded.append(self.alphabet[p])
            previous = p
        
        return ''.join(decoded)
    

In [7]:
import torchaudio
import torch

mel_spectrogram_converter = torchaudio.transforms.MelSpectrogram(
        sample_rate=16000,
        n_fft=400,
        n_mels=40
)

alphabet = Alphabet()

# from audio_utils
def preprocess_audio(batch):

    max_input_length = 0
    max_label_length = 0

    audio = []
    label = []
    audio_length = []
    label_length = []

    for item in batch:
        audio_array = torch.tensor(item['audio']['array']).float()

        sample_rate = item['audio']['sampling_rate']
        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(sample_rate, 16000)
            audio_array = resampler(audio_array)
        
        audio_array = torchaudio.functional.preemphasis(audio_array)

        mel_spectrogram = mel_spectrogram_converter(audio_array)

        # Apply log to the mel spectrogram
        mel_spectrogram = torch.log(mel_spectrogram + 1e-9)

        # Normalize the spectrogram
        mel_spectrogram = (mel_spectrogram - mel_spectrogram.mean()) / mel_spectrogram.std()

        # Transpose the mel spectrogram to correct dimension (time, mels)
        mel_spectrogram = mel_spectrogram.transpose(0, 1)

        # eka bugi löyty (väärä shape (1))
        max_input_length = max(max_input_length, mel_spectrogram.shape[0])

        sentence = torch.tensor(alphabet.text_to_array(str.lower(item['sentence'])))

        max_label_length = max(max_label_length, len(sentence))

        audio.append(mel_spectrogram)
        label.append(sentence)
        audio_length.append(mel_spectrogram.shape[0])
        label_length.append(len(sentence))

    audio_padded = map(lambda x: torch.nn.functional.pad(x, (0, 0, 0, max_input_length - x.size(0))), audio)

    blank_index = len(alphabet.alphabet) - 1  # Index of the blank token
    labels_padded = map(lambda x: torch.nn.functional.pad(x, (0, max_label_length - len(x)), value=blank_index), label)

    return (list(audio_padded), list(labels_padded), audio_length, label_length)


In [10]:
%pwd

'c:\\Users\\emila\\Documents\\koulu\\opinnaytetyo\\project\\notebooks'

In [11]:
import datasets
dataset = datasets.load_from_disk(dataset_path='../data/hf')
val_dataset = dataset['validation']


In [12]:
def load_custom_model(model_path, device):
    alphabet = Alphabet()
    model = LSTMCTC(
        input_size=40,
        hidden_size=320,
        num_layers=4,
        num_classes=len(alphabet.alphabet),
        dropout_rate=0.2
    ).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model, alphabet

In [13]:
def load_hf_model(model_name, device):
    processor = AutoProcessor.from_pretrained(model_name)
    model = AutoModelForSpeechSeq2Seq.from_pretrained(model_name).to(device)
    model.eval()
    return model, processor


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

custom_model_path = "../gd.pth"  # Adjust this path as needed
custom_model, custom_alphabet = load_custom_model(custom_model_path, device)

hf_model_name = "Finnish-NLP/whisper-large-finnish-v3"
hf_model, hf_processor = load_hf_model(hf_model_name, device)

Using device: cuda


C:\Users\emila\AppData\Local\Temp\ipykernel_26524\1431268298.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


In [21]:
sample = val_dataset[0]
sample_audio = torch.tensor(sample['audio']['array']).float().unsqueeze(0)
sample_text = sample['sentence']

print(f"Original text: {sample_text}", sample_audio.shape)

Original text: Hirvitys hyökkäsi sitä lähinnä olevan tankin kimppuun. torch.Size([1, 302400])


In [22]:
for i in range(20):
    print(val_dataset[i]['sentence'])

Hirvitys hyökkäsi sitä lähinnä olevan tankin kimppuun.
Äänestämme siis tämänsuuntaisten tarkistusten puolesta.
Kuin kimppuun ampuen.
Hän oli aina ollut niin taiteellinen, että arvasinkin, että hän voisi voittaa.
Avasin silmäni, ja aloin tokkurassa prosessoida sitä, mitä Tukuk oli juuri sanonut.
Ja siihen luottivat toisetkin.
Äänestimme tietenkin tätä kohtaa vastaan.
Äänestimme näin ollen nykyistä suositusta vastaan.
Ja sillä oli korvat päässä, punaiset.
Eikä rakkineessa ollut avaimellekaan paikkaa.
Äänestin direktiivin hylkäämistä koskevan tarkistuksen puolesta.
Äänestimme siis kyseisiä tarkistuksia vastaan ja koko tekstistä tyhjää.
Polutonta rinnettä, sakeaan viidakkoon.
Ylpeä se oli ollut kesällä.
Äänestän tätä tekstiä vastaan.
Äänestän näin ollen tyhjää, koska ehdotukseen sisältyy sekä kielteisiä että myönteisiä kohtia.
Lääkäri saapui paikalle.
Tuodakseen joukkoja maasta auttamaan vallankumouksessa Mysteerimies oli järjestänyt kymmenkunta sota-alusta maahan.
Kuljettaja löysi paikall

In [17]:
processed_sample = preprocess_audio([sample])[0][0].unsqueeze(0).to(device)

# Evaluate custom model
with torch.no_grad():
    custom_output = custom_model(processed_sample)
    custom_log_probs = F.log_softmax(custom_output, dim=-1)
    custom_decoded = custom_alphabet.decode(custom_log_probs, remove_blanks=True)[0]

print(f"Custom model output: {custom_decoded}")

Custom model output: hirvitys hyöppäsisitä lähiillä olevan tankinki ppuun


In [19]:
hf_input = hf_processor(sample_audio.numpy(), sampling_rate=16000, return_tensors="pt").input_features.to(device)

# Evaluate Hugging Face model
with torch.no_grad():
    generated_ids = hf_model.generate(input_features=hf_input, language="fi", task="transcribe")
    hf_decoded = hf_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(f"Hugging Face model output: {hf_decoded}")

Hugging Face model output: Hallituksen suurta astetta häntä ylövyn pönkkäämisen mukaan.


In [23]:
for i in range(20):
    gd = val_dataset[i]['sentence']
    audio = torch.tensor(val_dataset[i]['audio']['array']).float().unsqueeze(0)

    hf_input = hf_processor(audio.numpy(), sampling_rate=16000, return_tensors="pt").input_features.to(device)

    # Evaluate Hugging Face model
    with torch.no_grad():   
        generated_ids = hf_model.generate(input_features=hf_input, language="fi", task="transcribe")
        hf_decoded = hf_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print(f"Hugging Face model output: {hf_decoded}, gd {gd}")

Hugging Face model output: Hallituksen suurta astetta häntä ylövyn pönkkäämisen mukaan., gd Hirvitys hyökkäsi sitä lähinnä olevan tankin kimppuun.
Hugging Face model output: “Äästä, äsästä, vääsyy, pörsää, tykö, assusta, tyy, ässtä...”, gd Äänestämme siis tämänsuuntaisten tarkistusten puolesta.
Hugging Face model output: Venäjä, gd Kuin kimppuun ampuen.
Hugging Face model output: Halluakki rääkkii yhteen takia ja työsäätiä päinvieskuja pois., gd Hän oli aina ollut niin taiteellinen, että arvasinkin, että hän voisi voittaa.
Hugging Face model output: Yhdessä saa olla yhdenkytty yksi toisen sektorin yhdenkyttyä osuuksia., gd Avasin silmäni, ja aloin tokkurassa prosessoida sitä, mitä Tukuk oli juuri sanonut.
Hugging Face model output: Prosessi on yksi peruste., gd Ja siihen luottivat toisetkin.
Hugging Face model output: Rakennus on vahvistunut ja säästöt ovat vahvistuneet., gd Äänestimme tietenkin tätä kohtaa vastaan.
Hugging Face model output: Käärästä huippua, säätää silmää., gd Äänest

In [25]:
import numpy as np
for i in range(5):  # Test with 5 samples

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    hf_model_name = "Finnish-NLP/whisper-large-finnish-v3"
    hf_processor = AutoProcessor.from_pretrained(hf_model_name)
    hf_model = AutoModelForSpeechSeq2Seq.from_pretrained(hf_model_name).to(device)
    resampler = torchaudio.transforms.Resample(orig_freq=48000, new_freq=16000)

    print(f"\nSample {i+1}:")
    gd = val_dataset[i]['sentence']
    audio = torch.tensor(val_dataset[i]['audio']['array']).float()
    sr = val_dataset[i]['audio']['sampling_rate']
    
    print(f"Original audio shape: {audio.shape}, Sampling rate: {sr}")
    
    # Resample audio to 16000 Hz
    if sr != 16000:
        audio = resampler(audio)
        sr = 16000
    
    print(f"Resampled audio shape: {audio.shape}, New sampling rate: {sr}")
    print(f"Audio stats - Min: {torch.min(audio)}, Max: {torch.max(audio)}, Mean: {torch.mean(audio)}")
    
    hf_input = hf_processor(audio, sampling_rate=sr, return_tensors="pt").input_features.to(device)
    
    with torch.no_grad():
        output = hf_model.generate(
            input_features=hf_input,
            language="fi",
            task="transcribe",
            max_length=225,
            num_beams=5,
            output_scores=True,
            return_dict_in_generate=True
        )
        
    hf_decoded = hf_processor.batch_decode(output.sequences, skip_special_tokens=True)[0]
    
    print(f"Ground Truth: {gd}")
    print(f"HF Model Output: {hf_decoded}")
    
    # Print top 5 most likely tokens for the first 5 positions
    token_probs = torch.softmax(output.scores[0], dim=-1)
    top_tokens = torch.topk(token_probs[:5], k=5, dim=-1)
    for pos, (values, indices) in enumerate(zip(top_tokens.values, top_tokens.indices)):
        top_5 = [hf_processor.decode([idx.item()]) for idx in indices]
        print(f"Position {pos+1} top tokens: {', '.join(top_5)}")


Sample 1:
Original audio shape: torch.Size([302400]), Sampling rate: 48000
Resampled audio shape: torch.Size([100800]), New sampling rate: 16000
Audio stats - Min: -0.9065377116203308, Max: 0.9499269723892212, Mean: -0.00010964000830426812


c:\Users\emila\Documents\koulu\opinnaytetyo\project\env\Lib\site-packages\transformers\generation\configuration_utils.py:777: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_scores` is. When `return_dict_in_generate` is not `True`, `output_scores` is ignored.
  warnings.warn(
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


Ground Truth: Hirvitys hyökkäsi sitä lähinnä olevan tankin kimppuun.
HF Model Output: Hirvitys hyökkäsi sitä lähinnä olevan tankin kimppuun.
Position 1 top tokens: H, Hy, Hi,  Hir, Her

Sample 2:
Original audio shape: torch.Size([229824]), Sampling rate: 48000
Resampled audio shape: torch.Size([76608]), New sampling rate: 16000
Audio stats - Min: -0.8869452476501465, Max: 1.007727026939392, Mean: -3.974119681515731e-05
Ground Truth: Äänestämme siis tämänsuuntaisten tarkistusten puolesta.
HF Model Output: Äänestämme siis tämän suuntaisten tarkistusten puolesta.
Position 1 top tokens: �, A, Ö,  ä,  Ä

Sample 3:
Original audio shape: torch.Size([148608]), Sampling rate: 48000
Resampled audio shape: torch.Size([49536]), New sampling rate: 16000
Audio stats - Min: -0.7911140322685242, Max: 0.9635905027389526, Mean: -9.09962909645401e-05
Ground Truth: Kuin kimppuun ampuen.
HF Model Output: Kuin kimppuun ampuen.
Position 1 top tokens: K, ku, Qu, C,  K

Sample 4:
Original audio shape: torch.Si

KeyboardInterrupt: 